In [1]:
import pandas as pd

# Load the new file
df_master = pd.read_excel("pmn_master july 15 2025.xlsx")
df_transitions = pd.read_excel("Master Transition Planner.xlsx")
df_overview = pd.read_excel("Method Overview Dictionary.xlsx")

In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
import io
from datetime import datetime

# Load spreadsheet
df = pd.read_excel("pmn_master july 15 2025.xlsx")

# Clean and prepare data
df.rename(columns={
    'Series': 'series',
    'Book Title': 'book_title',
    'Book Order': 'book_order',
    'Concept': 'concept',
    'Intro Type': 'intro_type',
    'Review Status': 'review_status'
}, inplace=True)

df['concept'] = df['concept'].astype(str).str.strip().str.lower()
df['book_title'] = df['book_title'].astype(str).str.strip()
df.sort_values(by=['series', 'book_order'], inplace=True)
df.reset_index(drop=True, inplace=True)

weights_dict = df.groupby('concept')['Weight'].max().to_dict()

def build_summary_widget(df):
    nonlocal_export_filename = {"name": "comparison_results.xlsx"}
    download_output = widgets.Output()
    series_list = sorted(df['series'].unique())
    from_series_dropdown = widgets.Dropdown(options=series_list, description='From Series:')
    from_book_dropdown = widgets.Dropdown(description='From Book:')
    output = widgets.Output()
    download_button = widgets.Button(description="Download Excel", button_style='success', disabled=True)
    excel_data = io.BytesIO()

    def update_from_books(*args):
        series = from_series_dropdown.value
        books = df[df['series'] == series][['book_order', 'book_title']].drop_duplicates().sort_values('book_order')
        from_book_dropdown.options = [
            (f"{int(row.book_order)} – {row.book_title}", row.book_order)
            for row in books.itertuples(index=False)
        ]

    from_series_dropdown.observe(update_from_books, names='value')
    update_from_books()

    def on_compare_clicked(b):
        with output:
            clear_output()
            from_series = from_series_dropdown.value
            from_book = from_book_dropdown.value

            from_book_title = df.loc[
                (df['series'] == from_series) & (df['book_order'] == from_book),
                'book_title'
            ].values[0]

            from_label = f"{from_series} - {from_book_title}"
            date_str = datetime.today().strftime('%B %d %Y')
            export_filename = f"Comparison - {from_label} - {date_str}.xlsx".replace('/', '-')
            nonlocal_export_filename["name"] = export_filename

            current_concepts = set(df[(df['series'] == from_series) & (df['book_order'] <= from_book)]['concept'])

            summary_data = []

            for to_series in series_list:
                if to_series == from_series:
                    continue
            
                to_books = df[df['series'] == to_series]['book_order'].unique()
                for to_book in to_books:
                    to_df = df[(df['series'] == to_series) & (df['book_order'] == to_book)]
                    earlier_df = df[(df['series'] == to_series) & (df['book_order'] < to_book)]
                    to_title = to_df['book_title'].iloc[0] if not to_df.empty else ''
            
                    to_book_set = set(to_df['concept'])
                    to_earlier_set = set(earlier_df['concept'])
            
                    repeated = current_concepts & to_book_set
                    new = to_book_set - current_concepts
                    missing = to_earlier_set - current_concepts
            
                    def get_weighted_total(concepts):
                        return sum(weights_dict.get(c, 0.25) for c in concepts)
            
                    total_weight = get_weighted_total(to_book_set) or 1.0
            
                    repeated_weighted = get_weighted_total(repeated)
                    new_weighted = get_weighted_total(new)
            
                    pct_repeated_weighted = round(repeated_weighted / total_weight * 100, 1) if total_weight else 0
                    pct_new_weighted = round(new_weighted / total_weight * 100, 1) if total_weight else 0
            
                    def sort_by_weight(concepts):
                        return sorted(concepts, key=lambda c: weights_dict.get(c, 0.25), reverse=True)
            
                    summary_data.append({
                        'To Series': to_series,
                        'To Book': to_book,
                        'To Book Title': to_title,
                        '% Repeated (Weighted)': pct_repeated_weighted,
                        '% New (Weighted)': pct_new_weighted,
                        '# Missing': len(missing),
                        'Repeated Concepts': ", ".join(sort_by_weight(repeated)),
                        'New Concepts': ", ".join(sort_by_weight(new)),
                        'Missing Concepts': ", ".join(sort_by_weight(missing)),
                    })

            summary_df = pd.DataFrame(summary_data)
            

            # ----------- Natural Progression -----------
            next_books = df[(df['series'] == from_series) & (df['book_order'] > from_book)]
            if not next_books.empty:
                next_book_order = next_books['book_order'].min()
                next_book_df = df[(df['series'] == from_series) & (df['book_order'] == next_book_order)]

                next_concepts = set(next_book_df['concept'])
                repeated_np = current_concepts & next_concepts
                new_np = next_concepts - current_concepts

                repeated_np_count = len(repeated_np)
                new_np_count = len(new_np)
                total_np = repeated_np_count + new_np_count

                repeated_np_pct = round(repeated_np_count / total_np * 100) if total_np else 0
                new_np_pct = round(new_np_count / total_np * 100) if total_np else 0

                print("\nNatural Progression (Same Series → Next Book):")
                print(f"  Repeated: {repeated_np_count} concepts ({repeated_np_pct}%)")
                print(f"  New: {new_np_count} concepts ({new_np_pct}%)")

            display(summary_df)

            # ----------- Excel Export -----------
            excel_data.seek(0)
            excel_data.truncate()
            
            # Convert % columns to decimal fractions (e.g., 86 → 0.86)
            for col in summary_df.columns:
                if '%' in col:
                    summary_df[col] = summary_df[col] / 100
            
            with pd.ExcelWriter(excel_data, engine='xlsxwriter') as writer:
                workbook = writer.book
                percent_fmt = workbook.add_format({'num_format': '0%'})
                green_fmt = workbook.add_format({'bg_color': '#C6EFCE'})
                yellow_fmt = workbook.add_format({'bg_color': '#FFF2CC'})
                green_percent_fmt = workbook.add_format({'num_format': '0%', 'bg_color': '#C6EFCE'})
                yellow_percent_fmt = workbook.add_format({'num_format': '0%', 'bg_color': '#FFF2CC'})
            
                # Write comparison data to main sheet
                summary_df.to_excel(writer, index=False, sheet_name='Comparison')
                worksheet = writer.sheets['Comparison']
            
                for i, col in enumerate(summary_df.columns):
                    max_len = max(
                        summary_df[col].astype(str).map(len).max(),
                        len(str(col))
                    ) + 2
            
                    is_percent = '%' in col
                    is_weighted = 'Weighted' in col
                    is_unweighted = 'Unweighted' in col
            
                    if is_percent and is_weighted:
                        worksheet.set_column(i, i, max_len, green_percent_fmt)
                    elif is_percent and is_unweighted:
                        worksheet.set_column(i, i, max_len, yellow_percent_fmt)
                    elif is_weighted:
                        worksheet.set_column(i, i, max_len, green_fmt)
                    elif is_unweighted:
                        worksheet.set_column(i, i, max_len, yellow_fmt)
                    else:
                        worksheet.set_column(i, i, max_len)
            
                # Write natural progression data (if exists)
                if 'repeated_np_count' in locals() and 'new_np_count' in locals():
                    natural_df = pd.DataFrame([{
                        'Next Book (Same Series)': next_book_df['book_title'].iloc[0],
                        '# Repeated': repeated_np_count,
                        '% Repeated': repeated_np_pct / 100,
                        '# New': new_np_count,
                        '% New': new_np_pct / 100
                    }])
                    natural_df.to_excel(writer, index=False, sheet_name='Natural Progression')
                    np_sheet = writer.sheets['Natural Progression']
            
                    for i, col in enumerate(natural_df.columns):
                        max_len = max(
                            natural_df[col].astype(str).map(len).max(),
                            len(str(col))
                        ) + 2
            
                        if '%' in col:
                            np_sheet.set_column(i, i, max_len, percent_fmt)
                        else:
                            np_sheet.set_column(i, i, max_len)



            download_button.disabled = False

    def on_download_clicked(b):
        filepath = nonlocal_export_filename["name"]
        with open(filepath, "wb") as f:
            f.write(excel_data.getvalue())
        with download_output:
            clear_output()
            display(FileLink(filepath, result_html_prefix="�� Click to download: "))

    compare_button = widgets.Button(description="Compare", button_style='primary')
    compare_button.on_click(on_compare_clicked)

    download_button.on_click(on_download_clicked)

    ui = widgets.VBox([
        widgets.HBox([from_series_dropdown, from_book_dropdown]),
        compare_button,
        download_button,
        download_output,
        output
    ])

    display(ui)

# Run it
build_summary_widget(df)

In [3]:
# Copy this entire code block into a new cell in your Jupyter notebook

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import Dropdown, Button, VBox, HBox, Output, Checkbox, HTML, Accordion

# Load the data
df = pd.read_excel("pmn_master july 15 2025.xlsx")

# Clean and prepare data
df.rename(columns={
    'Series': 'series',
    'Book Title': 'book_title',
    'Book Order': 'book_order',
    'Concept': 'concept',
    'Intro Type': 'intro_type',
    'Review Status': 'review_status'
}, inplace=True)

df['concept'] = df['concept'].astype(str).str.strip().str.lower()
df['book_title'] = df['book_title'].astype(str).str.strip()
df.sort_values(by=['series', 'book_order'], inplace=True)
df.reset_index(drop=True, inplace=True)

weights_dict = df.groupby('concept')['Weight'].max().to_dict()

def compare_books_cumulative_fixed(df, current_series, current_book_order, new_series, new_book_order, include_reviews=True):
    """
    Updated logic for Widget 2 (drilldown):
    - from_set: all concepts in current_series up to and including current_book_order
    - to_book_set: only concepts in new_series, new_book_order
    - to_earlier_set: all concepts in new_series, book_order < new_book_order
    - repeated = from_set & to_book_set
    - new = to_book_set - from_set
    - missing = to_earlier_set - from_set
    - Percentages: weighted, denominator is total weight of to_book_set (repeated + new)
    - Missing: raw count
    """
    from_set = set(df[(df['series'] == current_series) & (df['book_order'] <= current_book_order)]['concept'])
    to_book_set = set(df[(df['series'] == new_series) & (df['book_order'] == new_book_order)]['concept'])
    to_earlier_set = set(df[(df['series'] == new_series) & (df['book_order'] < new_book_order)]['concept'])

    repeated = from_set & to_book_set
    new = to_book_set - from_set
    missing = to_earlier_set - from_set

    def get_weighted_total(concepts):
        return sum(weights_dict.get(c, 0.25) for c in concepts)

    total_weight = get_weighted_total(to_book_set) or 1.0
    repeated_weighted = get_weighted_total(repeated)
    new_weighted = get_weighted_total(new)

    pct_repeated_weighted = round(repeated_weighted / total_weight * 100, 1) if total_weight else 0
    pct_new_weighted = round(new_weighted / total_weight * 100, 1) if total_weight else 0

    def get_details(concepts, series, book_order):
        subset = df[(df['series'] == series) & (df['book_order'] <= book_order)]
        details = {}
        for concept in concepts:
            match = subset[subset['concept'] == concept]
            if not match.empty:
                row = match.iloc[0]
                details[concept] = {
                    'book_title': row['book_title'],
                    'page': row.get('Page', 'N/A'),
                    'intro_type': row.get('intro_type', 'N/A'),
                    'review_status': row.get('review_status', 'N/A')
                }
        return details

    repeated_d = get_details(repeated, new_series, new_book_order)
    new_d = get_details(new, new_series, new_book_order)
    missing_d = get_details(missing, new_series, new_book_order - 1)

    def sort_by_weight(concepts):
        return sorted(concepts, key=lambda c: weights_dict.get(c, 0.25), reverse=True)

    return {
        'repeated': repeated_d,
        'new': new_d,
        'missing': missing_d,
        'percentages': {
            'repeated_count': len(repeated),
            'new_count': len(new),
            'missing_count': len(missing),
            'repeated_pct_weighted': pct_repeated_weighted,
            'new_pct_weighted': pct_new_weighted,
            'total_weight': total_weight
        },
        'lists': {
            'repeated': sort_by_weight(repeated),
            'new': sort_by_weight(new),
            'missing': sort_by_weight(missing)
        }
    }
       
    
    repeated_d = get_details(repeated, new_series, new_book_order)
    new_d = get_details(new, new_series, new_book_order)
    missing_d = get_details(missing, new_series, new_book_order - 1)
    
    total = len(repeated | new | missing)
    pct = lambda x: round((x / total) * 100, 1) if total > 0 else 0
    
    # Calculate weighted values
    def get_weighted_total(concepts):
        return sum(weights_dict.get(c, 0.25) for c in concepts)
    
    total_weight = get_weighted_total(repeated | new | missing) or 1.0
    repeated_weighted = get_weighted_total(repeated)
    new_weighted = get_weighted_total(new)
    missing_weighted = get_weighted_total(missing)
    
    pct_repeated_weighted = round(repeated_weighted / total_weight * 100) if total_weight > 0 else 0
    pct_new_weighted = round(new_weighted / total_weight * 100) if total_weight > 0 else 0
    pct_missing_weighted = round(missing_weighted / total_weight * 100) if total_weight > 0 else 0
    
    return {
        'repeated': repeated_d,
        'new': new_d,
        'missing': missing_d,
        'percentages': {
            'repeated_count': len(repeated),
            'new_count': len(new),
            'missing_count': len(missing),
            'repeated_pct': pct(len(repeated)),
            'new_pct': pct(len(new)),
            'missing_pct': pct(len(missing)),
            'total_concepts': total,
            # Add weighted percentages
            'repeated_weighted': repeated_weighted,
            'new_weighted': new_weighted,
            'missing_weighted': missing_weighted,
            'repeated_pct_weighted': pct_repeated_weighted,
            'new_pct_weighted': pct_new_weighted,
            'missing_pct_weighted': pct_missing_weighted,
            'total_weight': total_weight
        }
    }

def build_working_widget(df, weights_dict):
    """
    WORKING WIDGET - Simple, reliable approach
    """
    
    series_options = sorted(df['series'].unique())
    
    def get_book_options(series):
        if not series:
            return []
        books = df[df['series'] == series][['book_order', 'book_title']].drop_duplicates().sort_values('book_order')
        return [(f"{int(row.book_order)} – {row.book_title}", int(row.book_order)) for row in books.itertuples(index=False)]
    
    # Create widgets
    current_series_dd = Dropdown(
        options=series_options, 
        value=series_options[0] if series_options else None,
        description='Current Series:', 
        layout={'width': '300px'}
    )
    
    new_series_dd = Dropdown(
        options=series_options, 
        value=series_options[1] if len(series_options) > 1 else series_options[0] if series_options else None,
        description='New Series:', 
        layout={'width': '300px'}
    )
    
    # Initialize book dropdowns
    current_book_options = get_book_options(series_options[0] if series_options else None)
    current_book_dd = Dropdown(
        options=current_book_options,
        value=current_book_options[0][1] if current_book_options else None,
        description='Current Book:', 
        layout={'width': '300px'}
    )
    
    new_book_options = get_book_options(series_options[1] if len(series_options) > 1 else series_options[0] if series_options else None)
    new_book_dd = Dropdown(
        options=new_book_options,
        value=new_book_options[0][1] if new_book_options else None,
        description='New Book:', 
        layout={'width': '300px'}
    )
    
    output = Output()
    
    def update_current_books(change):
        if change['new']:
            books = get_book_options(change['new'])
            if books:
                current_book_dd.options = books
                current_book_dd.value = books[0][1]
            else:
                current_book_dd.options = []
                current_book_dd.value = None
    
    def update_new_books(change):
        if change['new']:
            books = get_book_options(change['new'])
            if books:
                new_book_dd.options = books
                new_book_dd.value = books[0][1]
            else:
                new_book_dd.options = []
                new_book_dd.value = None
    
    # Attach observers
    current_series_dd.observe(update_current_books, names='value')
    new_series_dd.observe(update_new_books, names='value')
    
    def run_drilldown(*args):
        output.clear_output()
        with output:
            print(f"Running drilldown with: {current_series_dd.value}, {current_book_dd.value}, {new_series_dd.value}, {new_book_dd.value}")
            
            if not all([current_series_dd.value, current_book_dd.value, 
                       new_series_dd.value, new_book_dd.value]):
                print("Please select all series and books.")
                return
            
            result = compare_books_cumulative_fixed(
                df,
                current_series_dd.value,
                current_book_dd.value,
                new_series_dd.value,
                new_book_dd.value
            )
            
            checkboxes = {'Repeated': [], 'New': [], 'Missing': []}
            concept_details = {'Repeated': result['repeated'], 'New': result['new'], 'Missing': result['missing']}
            
            # Generate checkbox groups with concepts sorted by weight (highest first)
            for group_name in ['Repeated', 'New', 'Missing']:
                # Get concepts and their weights for this group
                group_concepts = list(concept_details[group_name].keys())
                
                # Sort concepts by weight (highest first)
                def get_concept_weight(concept):
                    return weights_dict.get(concept, 0.25)
                
                sorted_concepts = sorted(group_concepts, key=get_concept_weight, reverse=True)
                
                # Create checkboxes in weight order
                for concept in sorted_concepts:
                    details = concept_details[group_name][concept]
                    weight = get_concept_weight(concept)
                    label = f"{concept} (weight: {weight:.2f}, {details.get('intro_type', '')}, {details.get('review_status', '')})"
                    cb = Checkbox(value=True, description=label, indent=False)
                    checkboxes[group_name].append(cb)
            
            percentage_html = HTML()
            
            def update_percentages(change=None):
                # Only include Repeated and New in the denominator
                included = {
                    group: [cb for cb in checkboxes[group] if cb.value]
                    for group in ['Repeated', 'New', 'Missing']
                }
                # Unweighted denominator: only repeated + new
                total = len(included['Repeated']) + len(included['New'])
                pct = lambda x: round((len(x) / total) * 100, 1) if total else 0
            
                # Weighted denominator: only repeated + new
                def get_weighted_total(concepts):
                    return sum(weights_dict.get(c, 0.25) for c in concepts)
            
                included_repeated = [cb.description.split(' (weight:')[0] for cb in included['Repeated']]
                included_new = [cb.description.split(' (weight:')[0] for cb in included['New']]
                included_missing = [cb.description.split(' (weight:')[0] for cb in included['Missing']]
            
                total_weight = get_weighted_total(included_repeated + included_new) or 1.0
                pct_weighted = lambda concepts: round((get_weighted_total(concepts) / total_weight) * 100, 1) if total_weight > 0 else 0
            
                percentage_html.value = (
                    f"<b>Updated Totals:</b><br>"
                    f"✅ Repeated: {len(included['Repeated'])} ({pct(included['Repeated'])}%) - Weighted: {pct_weighted(included_repeated)}%<br>"
                    f"🆕 New: {len(included['New'])} ({pct(included['New'])}%) - Weighted: {pct_weighted(included_new)}%<br>"
                    f"❌ Missing: {len(included['Missing'])} (count only, not included in %)"
                )
            
            for group in ['Repeated', 'New', 'Missing']:
                for cb in checkboxes[group]:
                    cb.observe(update_percentages, names='value')
            
            update_percentages()
            
            accordion = Accordion(children=[
                VBox(checkboxes['Repeated']),
                VBox(checkboxes['New']),
                VBox(checkboxes['Missing'])
            ])
            accordion.set_title(0, f"✅ Repeated ({len(checkboxes['Repeated'])} - {result['percentages']['repeated_pct_weighted']:.1f}% weighted)")
            accordion.set_title(1, f"🆕 New ({len(checkboxes['New'])} - {result['percentages']['new_pct_weighted']:.1f}% weighted)")
            accordion.set_title(2, f"❌ Missing ({len(checkboxes['Missing'])})")
            
            display(HTML(
                f"<b>From:</b> {current_series_dd.value}, Book {current_book_dd.value}<br>"
                f"<b>To:</b> {new_series_dd.value}, Book {new_book_dd.value}<br><br>"
                f"<b>Summary:</b><br>"
                f"Weighted - Repeated: {result['percentages']['repeated_count']} ({result['percentages']['repeated_pct_weighted']}%), "
                f"New: {result['percentages']['new_count']} ({result['percentages']['new_pct_weighted']}%)<br>"
                f"Missing: {result['percentages']['missing_count']}<br><br>"
        
            ))
            display(percentage_html)
            display(accordion)
    
    compare_btn = Button(description="Run Drilldown", button_style='success')
    compare_btn.on_click(run_drilldown)
    
    # Create the main widget container
    ui = VBox([
        HBox([current_series_dd, current_book_dd]),
        HBox([new_series_dd, new_book_dd]),
        compare_btn,
        output
    ])
    
    display(ui)

# Run the working widget
print("Loading data and creating WORKING widget...")
build_working_widget(df, weights_dict)

Loading data and creating WORKING widget...


In [4]:
import io
import base64
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

def build_structured_concept_explorer(df):
    all_concepts = sorted(df['concept'].unique())

    concept_selector = widgets.SelectMultiple(
        options=all_concepts,
        description='Concepts:',
        layout=widgets.Layout(width='500px', height='300px')
    )

    run_button = widgets.Button(description="Explore", button_style='info')
    output = widgets.Output()

    def run_explorer(*args):
        with output:
            clear_output()
            selected = list(concept_selector.value)
            if not selected:
                print("Please select one or more concepts.")
                return

            filtered = df[df['concept'].isin(selected)].copy()

            if 'Page' in filtered.columns:
                filtered['page'] = filtered['Page']
            else:
                filtered['page'] = 'N/A'

            def format_concept(row):
                concept_name = row['concept']
                intro_type = str(row.get('intro_type', '')).lower()
                review_status = str(row.get('review_status', '')).lower()
                if intro_type == 'informal':
                    annotation = f" (informal, {review_status})"
                    return concept_name + annotation
                return concept_name

            filtered['concept_label'] = filtered.apply(format_concept, axis=1)

            group = (
                filtered.groupby(['series', 'book_order', 'Page'])['concept_label']
                .apply(lambda x: ', '.join(sorted(set(x))))
                .reset_index()
            )

            pivot = group.pivot_table(
                index=['book_order', 'Page'],
                columns='series',
                values='concept_label',
                aggfunc='first',
                fill_value=""
            ).reset_index()

            display(pivot)

            # ---- Updated: Multi-Series Unique/Missing Concept Summary Table ----
            series_list = [col for col in pivot.columns if col not in ['book_order', 'Page']]
            summary_rows = []

            for series in series_list:
                # Concepts in this series
                this_concepts = set(filtered[filtered['series'] == series]['concept'])

                # Concepts in all *other* series
                other_concepts = set(filtered[filtered['series'].isin([s for s in series_list if s != series])]['concept'])

                unique_concepts = sorted(this_concepts - other_concepts)
                missing_concepts = sorted(other_concepts - this_concepts)

                summary_rows.append({
                      'Series': series,
                    'Unique Concepts (Selected)': ", ".join(unique_concepts),
                    'Missing Concepts (Selected)': ", ".join(missing_concepts),
                })

            summary_df = pd.DataFrame(summary_rows)

            print("\nConcept Coverage Summary:")
            display(summary_df)
            # ---- Export Both Tables as Downloadable CSVs ----
            # Pivot Table
            csv_buffer_pivot = io.StringIO()
            pivot.to_csv(csv_buffer_pivot, index=False)
            b64_pivot = base64.b64encode(csv_buffer_pivot.getvalue().encode()).decode()
            href_pivot = f'<a download="concept_explorer_output.csv" href="data:text/csv;base64,{b64_pivot}" target="_blank">⬇️ Download Concept Explorer Table</a>'

            # Summary Table
            csv_buffer_summary = io.StringIO()
            summary_df.to_csv(csv_buffer_summary, index=False)
            b64_summary = base64.b64encode(csv_buffer_summary.getvalue().encode()).decode()
            href_summary = f'<a download="concept_coverage_summary.csv" href="data:text/csv;base64,{b64_summary}" target="_blank">⬇️ Download Concept Coverage Summary</a>'

            # Display both
            display(HTML(f"{href_pivot}<br>{href_summary}"))
            # ---------------------------------------------------


    run_button.on_click(run_explorer)

    ui = widgets.VBox([
        concept_selector,
        run_button,
        output
    ])
    display(ui)


In [5]:
build_structured_concept_explorer(df)


In [8]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
import io
import json

# --- File selectors (text, not upload) ---
transition_file = widgets.Text(
    value="Master Transition Planner.xlsx",
    description="Transition File:",
    layout=widgets.Layout(width="400px")
)
overview_file = widgets.Text(
    value="Method Overview Dictionary.xlsx",
    description="Overview File:",
    layout=widgets.Layout(width="400px")
)
run_button = widgets.Button(description="Run Transition Analysis", button_style="success")
output = widgets.Output()
download_output = widgets.Output()

# --- Load PMN master (already in directory) ---
df_master = pd.read_excel("pmn_master july 15 2025.xlsx")
df_master.rename(columns={
    'Series': 'series',
    'Book Title': 'book_title',
    'Book Order': 'book_order',
    'Concept': 'concept',
    'Weight': 'weight'
}, inplace=True)
df_master['concept'] = df_master['concept'].astype(str).str.strip().str.lower()
df_master['book_title'] = df_master['book_title'].astype(str).str.strip()
df_master.sort_values(by=['series', 'book_order'], inplace=True)
df_master.reset_index(drop=True, inplace=True)
weights_dict = df_master.groupby('concept')['weight'].max().to_dict()

# --- Helper for reading approach ---
def get_reading_approach(series, overview_df):
    if 'Series' not in overview_df.columns:
        return ""
    row = overview_df[overview_df['Series'].str.strip().str.lower() == str(series).strip().lower()]
    if not row.empty:
        return row.iloc[0]['Reading Approach']
    return ""

# --- Store the latest result globally for download ---
latest_df_result = {}

# --- Main logic ---
def run_transition_analysis(b):
    global latest_df_result
    with output:
        clear_output()
        try:
            df_transitions = pd.read_excel(transition_file.value)
            df_overview = pd.read_excel(overview_file.value)
        except Exception as e:
            print(f"Error loading files: {e}")
            return

        # Clean up column names (remove leading/trailing spaces)
        df_transitions.columns = [col.strip() for col in df_transitions.columns]
        df_overview.columns = [col.strip() for col in df_overview.columns]

        results = []
        for idx, row in df_transitions.iterrows():
            from_series = row['From Series']
            from_book = row['From Book Order']
            to_series = row['To Series']
            to_book_title = str(row['To Book']).strip()

            # Look up the book order for the to_book_title in the master file
            to_book_row = df_master[
                (df_master['series'] == to_series) &
                (df_master['book_title'].str.strip() == to_book_title)
            ]
            if not to_book_row.empty:
                to_book_order = to_book_row.iloc[0]['book_order']
            else:
                continue  # Skip this row silently if not found

            # Build sets
            from_set = set(df_master[
                (df_master['series'] == from_series) &
                (df_master['book_order'] <= from_book)
            ]['concept'])
            to_book_set = set(df_master[
                (df_master['series'] == to_series) &
                (df_master['book_order'] == to_book_order)
            ]['concept'])
            to_earlier_set = set(df_master[
                (df_master['series'] == to_series) &
                (df_master['book_order'] < to_book_order)
            ]['concept'])

            repeated = from_set & to_book_set
            new = to_book_set - from_set
            missing = to_earlier_set - from_set

            def get_weighted_total(concepts):
                return sum(weights_dict.get(c, 0.25) for c in concepts)

            total_weight = get_weighted_total(to_book_set) or 1.0
            repeated_weighted = get_weighted_total(repeated)
            new_weighted = get_weighted_total(new)
            pct_repeated = round(repeated_weighted / total_weight * 100, 1) if total_weight else 0
            pct_new = round(new_weighted / total_weight * 100, 1) if total_weight else 0

            def sort_by_weight(concepts):
                return sorted(concepts, key=lambda c: weights_dict.get(c, 0.25), reverse=True)

            # Reading approach
            from_approach = get_reading_approach(from_series, df_overview)
            to_approach = get_reading_approach(to_series, df_overview)

            results.append({
                'From Series': from_series,
                'From Book': row.get('From Book', ''),
                'From Book Order': from_book,
                'To Series': to_series,
                'To Book': to_book_title,
                'To Book Order': to_book_order,
                'Pacing Label': row.get('Pacing Label', ''),
                'Notes': row.get('Notes', ''),
                '% New (Weighted)': pct_new,
                '% Repeated (Weighted)': pct_repeated,
                '# Missing': len(missing),
                'New Concepts': ", ".join(sort_by_weight(new)),
                'Repeated Concepts': ", ".join(sort_by_weight(repeated)),
                'Missing Concepts': ", ".join(sort_by_weight(missing)),
                'From Reading Approach': from_approach,
                'To Reading Approach': to_approach
            })

        df_result = pd.DataFrame(results)
        latest_df_result['df'] = df_result  # Store for download
        display(df_result)

        # Show download links
        with download_output:
            clear_output()
            # CSV
            df_result.to_csv("transition_analysis.csv", index=False)
            # Excel
            df_result.to_excel("transition_analysis.xlsx", index=False)
            # JSON
            fields = [
                'From Series', 'From Book', 'From Book Order', 'To Series', 'To Book', 'To Book Order',
                'Pacing Label', 'Notes', '% New (Weighted)', '% Repeated (Weighted)', '# Missing',
                'New Concepts', 'Repeated Concepts', 'Missing Concepts', 'From Reading Approach', 'To Reading Approach'
            ]
            records = df_result[fields].to_dict(orient='records')
            with open("transition_analysis.json", "w", encoding="utf-8") as f:
                json.dump(records, f, ensure_ascii=False, indent=2)
            # JS variable
            with open("transition_analysis.js", "w", encoding="utf-8") as f:
                f.write("const transitionData = ")
                json.dump(records, f, ensure_ascii=False, indent=2)
                 # Uncomment for semicolon, leave off for none
            display(FileLink("transition_analysis.csv"))
            display(FileLink("transition_analysis.xlsx"))
            display(FileLink("transition_analysis.json"))
            display(FileLink("transition_analysis.js"))

# --- UI Layout ---
run_button.on_click(run_transition_analysis)
ui = widgets.VBox([
    widgets.HBox([transition_file, overview_file]),
    run_button,
    download_output,
    output
])
display(ui)